# data manipulation

## setting up

### installs & imports

In [33]:
! pip install openpyxl
! pip install pandas
! pip install petl

In [34]:
import openpyxl
import pandas as pd
import petl as etl

### Expenses Retrieval & Prep

In [35]:
# acquire the expense file
expenses_file = pd.read_excel('data/other/branch_expenses.xlsx')

# group via the county and acquire the sum
county_expenses = expenses_file.groupby(['county']).sum()


# create a new column which contains the total expenses
county_expenses['total_expenses'] = county_expenses['operational_cost'] \
                                    + county_expenses['staff_bonuses'] \
                                    + county_expenses['misc_expenses'] \
                                    + county_expenses['waste_cost']

# reset the index
county_expenses = county_expenses.reset_index()

### Complete Set of Records Retrieval & Prep

In [36]:
# majority of path and file name
incomplete_path = 'data/refined/branches_established_in_'

# one large df containing all cleaned data entries
complete_set = pd.read_csv(f'{incomplete_path}2010.csv').append(\
            pd.read_csv(f'{incomplete_path}2011.csv'), ignore_index=True).append(\
            pd.read_csv(f'{incomplete_path}2012.csv'), ignore_index=True)

In [37]:
# container df for 5 best & worst products per county & region
best_and_worst_products_regional = pd.DataFrame()
best_and_worst_products_per_county = pd.DataFrame()

# container df for best & worst prod categories per county & region
best_and_worst_prod_categories_regional = pd.DataFrame()
best_and_worst_prod_categories_per_county = pd.DataFrame()

# container df for performance of regions
regional_performance = pd.DataFrame()
# container df for best & worst performed counties
best_and_worst_performed_counties = pd.DataFrame()

# container df for the best branches hourly sales
best_branches_hourly_sales = pd.DataFrame()

# container df for the most & least profitable branches
most_and_least_profitable_branches = pd.DataFrame()

### iterating & saving

In [38]:
for region in complete_set.region.unique():
    # create a temp df with current region
    current_region = complete_set[complete_set.region == region]

    # create a temp df grouping previous via the product, sort descendingly by quantity sold
    current_regions_products = current_region.groupby(['region', 'product']).\
                                                sum().sort_values('total_quantity_purchased', ascending=False)

    # append the first and last 5 entries to regional product df
    best_and_worst_products_regional = best_and_worst_products_regional.\
                                        append(current_regions_products.head(5).\
                                        append(current_regions_products.tail(5)))

    # create a temp df grouping current region by prod category, sorted descendingly by quantity sold
    current_regions_prod_categorys = current_region.groupby(['region', 'product_category']).\
                                                        sum().sort_values('total_quantity_purchased', ascending=False)

    # append the current regions prod categories sales to regional prod category df
    best_and_worst_prod_categories_regional = best_and_worst_prod_categories_regional.\
                                                append(current_regions_prod_categorys)

    # create a temp df grouped by region
    current_regions_performance = current_region.groupby('region').sum()

    # append the current regions performance to regional performance df
    regional_performance = regional_performance.append(current_regions_performance)

    for county in current_region.county.unique():
        # create a temp df with current county
        current_county = current_region[current_region.county == county]

        # create a temp df grouping previous via the product, sort descendingly by quantity sold
        current_counties_products = current_county.groupby(['county', 'product']).\
                                                sum().sort_values('total_quantity_purchased', ascending=False)

        # append the first and last 5 entries to all counties product df
        best_and_worst_products_per_county = best_and_worst_products_per_county.\
                                        append(current_counties_products.head(5).\
                                        append(current_counties_products.tail(5)))

        # create a temp df grouping current county by prod category, sorted descendingly by quantity sold
        current_counties_prod_categorys = current_county.groupby(['county', 'product_category']).\
                                                        sum().sort_values('total_quantity_purchased', ascending=False)

        # append the current counties prod categories sales to all counties prod category df
        best_and_worst_prod_categories_per_county = best_and_worst_prod_categories_per_county.\
                                                append(current_counties_prod_categorys)

        # create a temp df grouped by county
        current_counties_performance = current_county.groupby('county').sum()

        # append the current counties performance to per county performance df
        best_and_worst_performed_counties = best_and_worst_performed_counties.append(current_counties_performance)


# thin down performed counties df to top and bottom 10 after being sorted
best_and_worst_performed_counties = best_and_worst_performed_counties.\
                                        sort_values(['total_quantity_purchased', 'amount_in_gbp'], ascending=False)

best_and_worst_performed_counties = best_and_worst_performed_counties.head(10).\
                                        append(best_and_worst_performed_counties.tail(10))

In [39]:
# acquire the best performed counties
list_of_best_counties = list(best_and_worst_performed_counties.head(10).reset_index().county)

# fill empty best_branches_hourly sales df with only those counties grouped by each hour and sorted
best_branches_hourly_sales = complete_set[complete_set.county.isin(list_of_best_counties)].\
                                groupby(['region', 'county', 'hour']).sum().\
                                sort_values(['county', 'hour'], ascending=True)

In [40]:
# acquire list of total expenses for each county which has been sorted previously via county name
list_of_expenses = list(county_expenses.total_expenses)

# create temp df from counties of complete set
temp_counties_df = complete_set.groupby(['region', 'county']).sum().reset_index()

# add new column which is each counties expense
temp_counties_df['total_expenses'] = list_of_expenses

# add another column called profits which is a sum of two other cols
temp_counties_df['total_profits'] = temp_counties_df['amount_in_gbp'] - temp_counties_df['total_expenses']

# sort temporary df based on total_profits
temp_counties_df = temp_counties_df.sort_values('total_profits')

# append top and bottom 10 profitable counties to empty df
most_and_least_profitable_branches = most_and_least_profitable_branches.\
                                        append(temp_counties_df.head(10).\
                                        append(temp_counties_df.tail(10)))

### saving

In [41]:
# file path for files
file_path = 'data/refined/results/'
# saving products per region
best_and_worst_products_regional.to_csv(f'{file_path}products_regional.csv')
# saving products per county
best_and_worst_products_per_county.to_csv(f'{file_path}products_per_county.csv')
# saving prod category per region
best_and_worst_prod_categories_regional.to_csv(f'{file_path}prod_categories_regional.csv')
# save prod category per county
best_and_worst_prod_categories_per_county.to_csv(f'{file_path}prod_categories_per_county.csv')
# save regional performance
regional_performance.to_csv(f'{file_path}regional_performance.csv')
# save per county performance
best_and_worst_performed_counties.to_csv(f'{file_path}top10_bottom10_performed_counties.csv')
# save best branches hourly sales
best_branches_hourly_sales.to_csv(f'{file_path}top10_counties_hourly_sales.csv')
# save profit results
most_and_least_profitable_branches.to_csv(f'{file_path}top10_bottom10_profitable_counties.csv', index=False)